# AXI Hardware Acceleration
This is a simple example showing the usage of a hardware accelerator. The accelerator multiplies two 32-bit numbers and returns its product after the enable signal has been asserted to one. The diagram below show the implementation of this architecture.

![Hardware Accelerator](AXI_accelerator.png)

The System Verilog code for the accelerator is

``` verilog
module accelerator(
    input logic clk, reset, enable,
    input logic [WIDTH-1 : 0] in_a, in_b,
    output logic [WIDTH-1 : 0] out);
    
    parameter WIDTH = 32;
    
    logic [2*WIDTH-1 : 0] temp;
    
    always_ff@(posedge clk)
        begin
        if(~reset) temp <= 0;
        else if(reset && enable) temp <= in_a * in_b;
        end
        
    assign out = temp[WIDTH-1 : 0];
    
endmodule

```

## Running the Hardware Accelerator Overlay

In the PYNQ-Z1 board upload into `/pynq/overlays/axi_accelerator` both the `bitstream` and `hardware handoff` files generated by Vivado. Then load your overlay

In [1]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/axi_accelerator/axi_accelerator_ver_1.bit')

Check if your overlay was correctly generated

In [2]:
overlay?

You should be able to see some information like this:
``` bash
IP Blocks
----------
axi_accelerator_0/S00_AXI : pynq.overlay.DefaultIP
axi_accelerator_0/S01_AXI : pynq.overlay.DefaultIP
processing_system7_0 : pynq.overlay.DefaultIP
```

In Python you will be able to access each AXI interface as different objects 

In [3]:
axi_lite = overlay.axi_accelerator_0.S00_AXI
axi_full = overlay.axi_accelerator_0.S01_AXI

First we set our enable signal to zero. Remember that the `enable` signal es mapped to `slv_reg0` from the AXI Lite interconnect.

In [4]:
axi_lite.write(0,0)

Now we set our memory mapped values with the AXI Full interconnect.

In [5]:
axi_full.write(0,25) #input in_a is mapped to address 0
axi_full.write(4,2) #input in_b is mapped to address 4

We enable the accelerator to start the calculation

In [6]:
axi_lite.write(0,1)

Now we check the correct result in the AXI Full interconnection at address zero.

In [7]:
axi_full.read(0)

50